## Library Imports

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')
pd.options.display.float_format = '{:.2f}'.format

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from typing import List, Tuple, Optional

from sklearn.pipeline import make_pipeline
import missingno as msno

In [2]:
# Load data
applications_history_data = pd.read_csv('data/applications_history.csv')
bki_data = pd.read_csv('data/bki.csv')
client_profile_data = pd.read_csv('data/client_profile.csv')
payments_data = pd.read_csv('data/payments.csv')
test_data = pd.read_csv('data/test.csv')
train_data = pd.read_csv('data/train.csv')

In [3]:
df_train = pd.merge(train_data, client_profile_data, how='left', on='APPLICATION_NUMBER', indicator='_MERGE_PROFILE')
df_test = pd.merge(test_data, client_profile_data, how='left', on='APPLICATION_NUMBER', indicator='_MERGE_PROFILE')

### Data cleaning

##### Drop Duplicate Observations

In [4]:
# Check for duplicates
print("Number of duplicates on the Train dataset:", df_train.duplicated().sum())
print("Number of duplicates on the Test dataset:", df_test.duplicated().sum())

Number of duplicates on the Train dataset: 0
Number of duplicates on the Test dataset: 0


##### Missing Observations

In [5]:
# Create new columns to flag features with a lot of missing values
flag_missing_columns = ['OWN_CAR_AGE', 'EXTERNAL_SCORING_RATING_1', 'EXTERNAL_SCORING_RATING_2',
                        'EXTERNAL_SCORING_RATING_3', 'AMT_REQ_CREDIT_BUREAU_MON']

for column in flag_missing_columns:
    df_train['MISSING_' + column] = ((df_train['_MERGE_PROFILE']=='both') & (df_train[column].isna())).astype('int')
    df_test['MISSING_' + column] = ((df_test['_MERGE_PROFILE']=='both') & (df_test[column].isna())).astype('int')
    
# Flag DAYS_ON_LAST_JOB > 350000
df_train['MISSING_DAYS_ON_LAST_JOB'] = (df_train.DAYS_ON_LAST_JOB > 350000).astype('int')
df_test['MISSING_DAYS_ON_LAST_JOB'] = (df_test.DAYS_ON_LAST_JOB > 350000).astype('int')

In [6]:
"""
# Fill out missing values in numeric features as median
df_train[df_train['MISSING_OWN_CAR_AGE']==1]['MISSING_OWN_CAR_AGE'] = 0

numeric_columns = df_train.drop(columns=['TARGET']).select_dtypes(include='number').columns
for column in numeric_columns:
    df_train[column].fillna(df_train[column].median(), inplace=True) 
    df_test[column].fillna(df_test[column].median(), inplace=True) 
"""

"\n# Fill out missing values in numeric features as median\ndf_train[df_train['MISSING_OWN_CAR_AGE']==1]['MISSING_OWN_CAR_AGE'] = 0\n\nnumeric_columns = df_train.drop(columns=['TARGET']).select_dtypes(include='number').columns\nfor column in numeric_columns:\n    df_train[column].fillna(df_train[column].median(), inplace=True) \n    df_test[column].fillna(df_test[column].median(), inplace=True) \n"

##### Process categorical features

In [7]:
features = df_train.drop(columns=['TARGET'])
target = df_train['TARGET']
features.shape

(110093, 32)

In [8]:
cat_features = features.select_dtypes(exclude='number').columns

In [9]:
df_train.GENDER.replace('XNA', 'Missing', inplace=True)
df_test.GENDER.replace('XNA', 'Missing', inplace=True)
df_train.FAMILY_STATUS.replace('Unknown', 'Missing', inplace=True)
df_test.FAMILY_STATUS.replace('Unknown', 'Missing', inplace=True)

In [10]:
#df_train = pd.get_dummies(df_train, cat_features)
#df_test = pd.get_dummies(df_test, cat_features)

In [11]:
#df_train.columns

##### Flag outliers in numeric features

In [12]:
# Flag outliers for 'TOTAL_SALARY', 'AMOUNT_CREDIT', 'AMOUNT_ANNUITY'
outliers_columns = ['TOTAL_SALARY', 'AMOUNT_CREDIT', 'AMOUNT_ANNUITY']

for column in outliers_columns:
    LEFT_BOUND_train, RIGHT_BOUND_train = np.percentile(df_train[column], q=1), np.percentile(df_train[column], q=99)
    LEFT_BOUND_test, RIGHT_BOUND_test = np.percentile(df_test[column], q=1), np.percentile(df_test[column], q=99)
    df_train['OUTLIER_' + column] = ((df_train[column] < LEFT_BOUND_train) | (df_train[column] > RIGHT_BOUND_train)).astype('int')
    df_test['OUTLIER_' + column] = ((df_test[column] < LEFT_BOUND_test) | (df_test[column] > RIGHT_BOUND_test)).astype('int')
    #df_train[column] = np.clip(df_train[column], LEFT_BOUND_train, RIGHT_BOUND_train)
    #df_test[column] = np.clip(df_test[column], LEFT_BOUND_test, RIGHT_BOUND_test)

##### Process numeric features

In [13]:
# Make CHILDRENS as a descrete/categorical feature
df_train['CHILDREN_0']  = (df_train.CHILDRENS == 0).astype('int')
df_train['CHILDREN_1']  = (df_train.CHILDRENS == 1).astype('int')
df_train['CHILDREN_2']  = (df_train.CHILDRENS == 2).astype('int')
df_train['CHILDREN_3+']  = (df_train.CHILDRENS >= 3).astype('int')
df_test['CHILDREN_0']  = (df_test.CHILDRENS == 0).astype('int')
df_test['CHILDREN_1']  = (df_test.CHILDRENS == 1).astype('int')
df_test['CHILDREN_2']  = (df_test.CHILDRENS == 2).astype('int')
df_test['CHILDREN_3+']  = (df_test.CHILDRENS >= 3).astype('int')

# Make FAMILY_SIZE as a descrete/categorical feature
df_train['FAMILY_SIZE_0']  = (df_train.FAMILY_SIZE == 0).astype('int')
df_train['FAMILY_SIZE_1']  = (df_train.FAMILY_SIZE == 1).astype('int')
df_train['FAMILY_SIZE_2']  = (df_train.FAMILY_SIZE == 2).astype('int')
df_train['FAMILY_SIZE_3+']  = (df_train.FAMILY_SIZE >= 3).astype('int')
df_test['FAMILY_SIZE_0']  = (df_test.FAMILY_SIZE == 0).astype('int')
df_test['FAMILY_SIZE_1']  = (df_test.FAMILY_SIZE == 1).astype('int')
df_test['FAMILY_SIZE_2']  = (df_test.FAMILY_SIZE == 2).astype('int')
df_test['FAMILY_SIZE_3+']  = (df_test.FAMILY_SIZE >= 3).astype('int')

#df_train.drop(columns=['CHILDRENS', 'FAMILY_SIZE'], inplace=True)

##### Generate new PROFILE metrics

In [14]:
# Generate financial metrics
df_train['AMOUNT_CREDIT_to_AMOUNT_ANNUITY'] = df_train['AMOUNT_CREDIT'] / df_train['AMOUNT_ANNUITY'] 
df_test['AMOUNT_CREDIT_to_AMOUNT_ANNUITY'] = df_test['AMOUNT_CREDIT'] / df_test['AMOUNT_ANNUITY'] 

df_train['AMOUNT_CREDIT_to_TOTAL_SALARY'] = df_train['AMOUNT_CREDIT'] / df_train['TOTAL_SALARY'] 
df_test['AMOUNT_CREDIT_to_TOTAL_SALARY'] = df_test['AMOUNT_CREDIT'] / df_test['TOTAL_SALARY'] 

df_train['AMOUNT_ANNUITY_to_TOTAL_SALARY'] = df_train['AMOUNT_ANNUITY'] / df_train['TOTAL_SALARY'] 
df_test['AMOUNT_ANNUITY_to_TOTAL_SALARY'] = df_test['AMOUNT_ANNUITY'] / df_test['TOTAL_SALARY'] 

df_train['TOTAL_SALARY_and_TOTAL_SALARY_diff'] = df_train['TOTAL_SALARY'] - df_train['AMOUNT_ANNUITY'] 
df_test['TOTAL_SALARY_and_TOTAL_SALARY_diff'] = df_test['TOTAL_SALARY'] - df_test['AMOUNT_ANNUITY'] 

In [15]:
# Generate scoring metrics
for function_name in ["mean", "nanmedian", 'min', 'max']:
    feature_name = "EXTERNAL_SCORING_RATINGS_{}".format(function_name)
    df_train[feature_name] = eval("np.{}".format(function_name))(
        df_train[["EXTERNAL_SCORING_RATING_1", "EXTERNAL_SCORING_RATING_2", "EXTERNAL_SCORING_RATING_3"]], axis=1
    )
    df_test[feature_name] = eval("np.{}".format(function_name))(
        df_test[["EXTERNAL_SCORING_RATING_1", "EXTERNAL_SCORING_RATING_2", "EXTERNAL_SCORING_RATING_3"]], axis=1
    )

/Users/sashas/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1113: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,


##### Generate new applications_history_data metrics

In [16]:
def create_numerical_aggs(data: pd.DataFrame,
                          groupby_id: str,
                          aggs: dict,
                          prefix: Optional[str] = None,
                          suffix: Optional[str] = None,
                          ) -> pd.DataFrame:
    if not prefix:
        prefix = ""
    if not suffix:
        suffix = ""

    data_grouped = data.groupby(groupby_id)
    stats = data_grouped.agg(aggs)
    stats.columns = [f"{prefix}{feature}_{stat}{suffix}".upper() for feature, stat in stats]
    stats = stats.reset_index()

    return stats

In [17]:
aggs = {
    'PREV_APPLICATION_NUMBER': ['count'],
    'AMT_APPLICATION': ['mean', 'min', 'max'],
    #'AMOUNT_CREDIT': ['mean', 'sum'],
    #'AMOUNT_ANNUITY': ['mean', 'sum'],
    #'AMOUNT_PAYMENT': ['sum'],
    #'AMOUNT_GOODS_PAYMENT': ['sum'],
    #"CNT_PAYMENT": ["mean"],
    'DAYS_DECISION': ['mean', 'min', 'max']

}

mask = applications_history_data["NAME_CONTRACT_STATUS"] == "Refused"
stats_approved = create_numerical_aggs(
    applications_history_data[mask], groupby_id="APPLICATION_NUMBER", aggs=aggs, prefix="PREV_REFUSED_"
)

df_train = pd.merge(df_train, stats_approved, how='left', on='APPLICATION_NUMBER')
df_test = pd.merge(df_test, stats_approved, how='left', on='APPLICATION_NUMBER')
df_train.head()

,APPLICATION_NUMBER,TARGET,NAME_CONTRACT_TYPE,GENDER,CHILDRENS,TOTAL_SALARY,AMOUNT_CREDIT,AMOUNT_ANNUITY,EDUCATION_LEVEL,FAMILY_STATUS,...,EXTERNAL_SCORING_RATINGS_nanmedian,EXTERNAL_SCORING_RATINGS_min,EXTERNAL_SCORING_RATINGS_max,PREV_REFUSED_PREV_APPLICATION_NUMBER_COUNT,PREV_REFUSED_AMT_APPLICATION_MEAN,PREV_REFUSED_AMT_APPLICATION_MIN,PREV_REFUSED_AMT_APPLICATION_MAX,PREV_REFUSED_DAYS_DECISION_MEAN,PREV_REFUSED_DAYS_DECISION_MIN,PREV_REFUSED_DAYS_DECISION_MAX
0,123687442,0,Cash,M,1.00,157500.00,855000.00,25128.00,Secondary / secondary special,Married,...,0.70,0.65,0.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,123597908,1,Cash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.00,495000.00,495000.00,495000.00,505.00,505.00,505.00
2,123526683,0,Cash,F,0.00,135000.00,1006920.00,42660.00,Higher education,Married,...,0.48,0.27,0.68,1.00,0.00,0.00,0.00,456.00,456.00,456.00
3,123710391,1,Cash,M,0.00,180000.00,518562.00,22972.50,Secondary / secondary special,Married,...,0.17,0.17,0.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,123590329,1,Cash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,6.00,372951.00,88353.00,697500.00,605.67,178.00,1461.00


In [18]:
"""
columns = ['PREV_REFUSED_PREV_APPLICATION_NUMBER_COUNT',
          'PREV_REFUSED_AMT_APPLICATION_MEAN',
          'PREV_REFUSED_AMT_APPLICATION_MIN',
          'PREV_REFUSED_AMT_APPLICATION_MAX',
          'PREV_REFUSED_DAYS_DECISION_MEAN',
          'PREV_REFUSED_DAYS_DECISION_MIN',
          'PREV_REFUSED_DAYS_DECISION_MAX']
for column in columns:
    df_train[column].fillna(0, inplace=True)
    df_test[column].fillna(0, inplace=True)
"""

"\ncolumns = ['PREV_REFUSED_PREV_APPLICATION_NUMBER_COUNT',\n          'PREV_REFUSED_AMT_APPLICATION_MEAN',\n          'PREV_REFUSED_AMT_APPLICATION_MIN',\n          'PREV_REFUSED_AMT_APPLICATION_MAX',\n          'PREV_REFUSED_DAYS_DECISION_MEAN',\n          'PREV_REFUSED_DAYS_DECISION_MIN',\n          'PREV_REFUSED_DAYS_DECISION_MAX']\nfor column in columns:\n    df_train[column].fillna(0, inplace=True)\n    df_test[column].fillna(0, inplace=True)\n"

In [19]:
df_train._MERGE_PROFILE = df_train._MERGE_PROFILE.astype('object')
df_test._MERGE_PROFILE = df_test._MERGE_PROFILE.astype('object', copy=False)

In [20]:
df_train.select_dtypes(exclude='number').dtypes

NAME_CONTRACT_TYPE    object
GENDER                object
EDUCATION_LEVEL       object
FAMILY_STATUS         object
_MERGE_PROFILE        object
dtype: object

In [21]:
# Fill out missing values in categorical features as Missing 
cat_columns = df_train.select_dtypes(exclude='number').columns
for column in cat_columns:
    df_train[column].fillna("Missing", inplace=True) 
    df_test[column].fillna("Missing", inplace=True) 

In [22]:
df_train.to_csv('data/df_train_missing_clean_catboost.csv', index=None)
df_test.to_csv('data/df_test_missing_clean_catboost.csv', index=None)